# Cross-Datasite Query Example: Bob Accessing Alice's Data

This notebook demonstrates how Bob can query Alice's data using remote LLM functionality. It assumes:
- Both Bob and Alice have already run their respective setup notebooks
- Alice has granted Bob permission to access specific files
- Bob now wants to analyze Alice's data without having direct access to it

In [1]:
from __future__ import annotations

import os
import time
from pathlib import Path
from loguru import logger
from tabulate import tabulate

# Add this to your notebook
from llama_remote import configure_logging

# Force DEBUG logging
configure_logging(verbose=True)

# Import the client module
from llama_remote import client

## 1. Connect as Bob

In [2]:
# Creating Bob's client
bob_client = client("~/.syft_bob_config.json")
print(f"Connected as: {bob_client.client.email}")

2025-03-09 04:06:34.531 | INFO     | syft_rpc_client:__init__:51 - 🔑 Connected as: bob@openmined.org
2025-03-09 04:06:34.532 | INFO     | syft_rpc_client:_run_server:69 - 🚀 SERVER: Running ollama_remote server as bob@openmined.org
2025-03-09 04:06:34.532 | INFO     | syft_rpc_client:_start_server:64 - 🔔 Server started for bob@openmined.org
2025-03-09 04:06:34.537 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /generate
2025-03-09 04:06:34.540 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /set_file_permissions
2025-03-09 04:06:34.543 | INFO     | syft_rpc_client:_run_server:93 - 📡 SERVER: Listening for requests at /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/ollama_remote/rpc
2025-03-09 04:06:34.547 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /list_file_permissions


Connected as: bob@openmined.org


2025-03-09 04:06:34.583 | INFO     | syft_event.server2:publish_schema:96 - Published schema to /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/ollama_remote/rpc/rpc.schema.json


In [3]:
# Test direct access to Alice's public directory
alice_public_dir = f"/Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/public"
print(f"Testing access to Alice's public directory: {alice_public_dir}")


# List files in the public directory
public_files = os.listdir(alice_public_dir)
print(f"Files in public directory: {public_files[:5]}{'...' if len(public_files) > 5 else ''}")


Testing access to Alice's public directory: /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/public
Files in public directory: ['iphone_2007.txt', 'syftperm.yaml']


In [4]:
# Request permissions list again with forced debug
print("\nRequesting permissions list with debug logging enabled:")
bob_client._check_file_permission(bob_client.client.email, f"{alice_public_dir}/some_file.txt")


2025-03-09 04:06:35.690 | DEBUG    | llama_remote:_check_file_permission:221 - Checking permissions for user bob@openmined.org on file /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/public/some_file.txt
2025-03-09 04:06:35.690 | DEBUG    | llama_remote:_check_file_permission:240 - No explicit permission found, checking standard Syft permissions
2025-03-09 04:06:35.691 | DEBUG    | llama_remote:_check_file_permission:247 - Datasite path: /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org
2025-03-09 04:06:35.691 | WARNING  | llama_remote:_check_file_permission:272 - File /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/public/some_file.txt is not within datasite path /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org



Requesting permissions list with debug logging enabled:


False

In [5]:
# Re-request the permissions list
permitted_files = bob_client.list_permitted_files(
    to_email="alice@openmined.org",
    user_email=bob_client.client.email
)
print(f"Updated permissions list: {permitted_files}")


2025-03-09 04:06:36.123 | INFO     | syft_rpc_client:send_request:160 - 📤 SENDING: Request to alice@openmined.org/list_file_permissions
2025-03-09 04:06:42.837 | INFO     | syft_rpc_client:send_request:175 - 📥 RECEIVED: Response from alice@openmined.org. Time: 6.71s


Updated permissions list: user_email='bob@openmined.org' allowed_files=['/Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/patient_data.csv'] success=True error=None ts=datetime.datetime(2025, 3, 9, 8, 6, 38, 474435, tzinfo=TzInfo(UTC))
